# PAYOFF MATRIX MODULE

This notebook implements the game-theoretical aspects of the code.  

In [1]:
#IMPORTS
#libraries
import numpy as np
import import_ipynb

#notebooks
import blockchain as bc

importing Jupyter notebook from blockchain.ipynb


In [2]:
#NOTE: we can check that the code returns the correct results for T<=3. we also did some checks for T=4, but not for all possibilities, just the ones that could likely contain errors (cases where there are multiple nash eq. off-path). 
#for all T>4 we have to assume the code returns correct results. 

## FUNCTIONS

The function $finalExpectedPayoff()$ makes use of the blockchain class method $getPayoff()$ to calculate the payoff for miner $i$ over the longest chains. That is, it computes the expected value for miner $i$ when Nature chooses one of the longest chains at random.   

The function $finalPayoffMatrix()$ builds on top of the previously mentioned $finalExpectedPayoff()$ function. It returns the payoff matrix of the normal-form simultaneous game in the final stage of the game. 

Next, we have the function $matrixNashEq()$. It finds the Nash equilibria of a payoff matrix. That payoff matrix may be generated by $finalPayoffMatrix()$ or $intermediatePayoffMatrix()$, both work. Finally, the function $getStrategies()$ returns the strategies of all miners in every Nash equilibrium.

NOTE: These functions only work for $n=2$ and are written with that assumption in mind. Going higher than $n=2$ is computationally unfeasible anyway.


In [3]:
def finalExpectedPayoff(_B, _i):
    #
    #'_B' is a blockchain object
    #'_i' is the index of the miner we want the compute the expected payoff for
    #
    #this function simply computes the expected payoff for miner 'i' over all the longest chains in '_B'
    #returns the payoff 'p' of the game if it were to end now (i.e. if '_B' defines the situation at the end of the game)
    #

    longest_chains = _B.longestchains #this is a list of lists!
    l = len(longest_chains) #number of longest chains
    payoffs = []

    #FILL THE LIST
    for chain in longest_chains:
        payoffs.append(_B.getPayoff(_i, chain[-1]))

    p = np.sum(payoffs) / l #each chain is chosen at random with equal probability

    return p



def finalPayoffMatrix(_B, _n, _t):
    #
    #calculates the payoff matrix of the normal form simultaneous game with '_n' players at stage '_t' with blockchain '_B'
    #before nature chooses the winning chain!
    #

    M = np.zeros((_t, _t, _n))

    for r in range(_t): #rows, blocks m0 can mine at
        for c in range(_t): #cols, blocks m1 can mine at

            #EXTEND THE CHAIN
            B_ext1 = bc.ExtendedBlockchain(_B, 0, r) #the case where miner 0 wins the final stage
            B_ext2 = bc.ExtendedBlockchain(_B, 1, c) #the case where miner 1 wins the final stage

            #EXPECTED VALUE
            M[r, c, 0] = 0.5 * (finalExpectedPayoff(B_ext1, 0) + finalExpectedPayoff(B_ext2, 0)) #payoff for miner 0
            M[r, c, 1] = 0.5 * (finalExpectedPayoff(B_ext1, 1) + finalExpectedPayoff(B_ext2, 1)) #payoff for miner 1

    return M



def matrixNashEq(_M):
    #
    #finds the Nash equilibrium of the payoff matrix '_M'
    #find mutual best responses!
    #returns a mask with 'True' where we have a Nash equilibrium
    #

    """
    t = _M.shape[0]

    best_responses0 = []
    best_responses1 = []

    #FIND BEST RESPONSES
    for i in range(t):
        row = _M[:, i, 0]
        col = _M[i, :, 1]
        best_responses0.append(row == row.max())
        best_responses1.append(col == col.max())

    nash_eq = np.full((_M.shape[0], _M.shape[1]), False)


    #CONFIGURE MASK
    for r in range(t): #rows
        for c in range(t): #cols
            if (best_responses0[c][r] == True) and (best_responses1[r][c] == True):
                nash_eq[r,c] = True
    """

    #EXTRACT PAYOFFS FOR EACH MINER
    M0 = _M[:,:,0] #payoff matrix for miner 0
    M1 = _M[:,:,1] #payoff matrix for miner 1

    #FIND BEST RESPONSES
    best_responses0 = (M0 == M0.max(axis=0, keepdims=True))
    best_responses1 = (M1 == M1.max(axis=1, keepdims=True))
    
    #CONFIGURE MASK
    nash_eq = np.logical_and(best_responses0, best_responses1)

    return nash_eq



def getStrategies(_B, _T, _n):
    #
    #determines the Nash equilibrium strategies for all '_n' miners given some blockchain '_B' and game horizon '_T'
    #returns a list of tuples containing the equilibirum strategies
    #nash_strats[0][1] is the strategy of the second [1] miner in the first [0] equilibirum
    #

    M = intermediatePayoffMatrix(_B, _T, _n)
    nash = matrixNashEq(M)
    nash_strats = np.transpose(np.where(nash == True))
    nash_strats = [tuple(element) for element in nash_strats] #convert to tuples

    return nash_strats

The final function of this module is $intermediatePayoffMatrix()$. It is the most complex function in this notebook, since it makes use of recursion. 

We make a *strong assumption* in our code. That is, we assume that cooperation is not necessary to play a Nash equilibrium and that each Nash equilibrium is played with equal probability. This assumption allows us to calculate the expected payoff as a simple probability weighted sum. We verify that this assumption is reasonable and holds for all finite blockchain games of length $T \leq 7$. The reader may consult the notebook labeled 'conjectures'.

The function is recursively called at every decision node in the game tree. This causes the code to take a long time to terminate because the game tree explodes very rapidly in size.

In [4]:
def intermediatePayoffMatrix(_B, _T, _n):
    #
    #calculates the payoff matrix for an intermediate stage of the multi-stage normal-form simultaneous game with '_n' players, 
    #which starts with blockchain '_B' and ends at stage '_T'
    #makes use of recursion!
    #

    t = _B.horizon + 1 #current stage, we build on top of '_B'
    strats = np.arange(t) #all possible strategies, that is choosing a block 'b_t'
    M = np.zeros((t, t, _n))

    if _T == t: #once we have reached the final stage
        M = finalPayoffMatrix(_B, _n, t)

    else: #otherwise we need backwards induction from the last stage 't = T' to get the payoffs for the current stage
        for strat0 in strats: #for every possible strategy of miner 0
            for strat1 in strats: #for every possible strategy of miner 1

                #EXTEND THE BLOCKCHAIN
                B_ext0 = bc.ExtendedBlockchain(_B, 0, strat0) #the case where miner 0 wins
                B_ext1 = bc.ExtendedBlockchain(_B, 1, strat1) #the case where miner 1 wins

                #CALL RECURSION
                #find the payoff matrices for the current stage, one for each possible winner
                M_ext0 = intermediatePayoffMatrix(B_ext0, _T, _n) #the case where miner 0 wins
                M_ext1 = intermediatePayoffMatrix(B_ext1, _T, _n) #the case where miner 1 wins

                #FIND NASH EQUILIBRIA
                #we are only interested in equilibira in pure strategies
                nash0 = matrixNashEq(M_ext0) #the case where miner 0 wins
                nash1 = matrixNashEq(M_ext1) #the case where miner 1 wins

                #FIND THE NUMBER OF NASH EQUILIBRIA
                n_eq0 = np.count_nonzero(nash0) #the case where miner 0 wins
                n_eq1 = np.count_nonzero(nash1) #the case where miner 1 wins

                #EXPECTED VALUE
                #we average over all Nash equilibria in the next stage to get the expected payoff in the current stage
                M[strat0, strat1, 0] = 0.5 * (M_ext0[nash0].sum(axis=0)[0]/n_eq0 + M_ext1[nash1].sum(axis=0)[0]/n_eq1) #expected payoff for miner 0
                M[strat0, strat1, 1]  = 0.5 * (M_ext0[nash0].sum(axis=0)[1]/n_eq0 + M_ext1[nash1].sum(axis=0)[1]/n_eq1) #expected payoff for miner 1

    return M